### Problem 6.4

Let us define our domain: 

In [3]:
import numpy as np 
import matplotlib.pyplot as plt 

L  = 100 
dx = 0.1 

x = np.arange(0, L+dx, dx)  

N = len(x) 

# Velocity (v) and stress (sigma)
v     = np.zeros(N)
sigma = np.zeros(N)

# Material properties
rho = np.zeros(N)
mu = np.zeros(N)

Define the initial condition as per the problem: 

In [7]:
def apply_initial_condition(x, v, sigma): 
    sigma[:] = 0
    v = -0.2 * (x-50) * np.exp(-0.1*(x-50)**2)
    return v, sigma